<a href="https://colab.research.google.com/github/yuifuku1118/TobyPy/blob/master/YY_roma_model_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install pykakasi
!pip install romkan
!pip install pandas 

#モデル作成

## データセット作成

In [ ]:
from google.colab import drive

/drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import os
def get_filename(pathname):
    return os.path.basename(pathname)

Yuichiro Model


In [ ]:
import glob
import pandas as pd

In [ ]:
file1 = open('/content/gdrive/MyDrive/yamaoka_san_voice/label.txt', 'r')
Lines = file1.readlines()

In [ ]:
pathdict = {}
for i in Lines:
    s = i.rstrip('\n').split('.')
    label = s[1].replace(" ", "")
    name = int(s[0])
    pathdict[name] = label

In [ ]:
pathdict

{1: 'お電話ありがとうございます',
 2: 'どのようなご用件',
 3: 'どのようなお問い合わせ',
 4: 'ありがとう',
 5: '申し訳ありません',
 6: 'はい',
 7: 'いいえ',
 8: '今後ともよろしくお願いいたします',
 9: '解決しました',
 10: '解決しません',
 11: 'よかったです',
 12: '本当に',
 13: 'おせわになっております',
 14: 'おはようございます',
 15: 'こんにちわ',
 16: 'いえ',
 17: 'そうですね',
 18: 'ありがとうございます',
 19: '助かりました',
 20: '大変助かりました',
 21: 'おかしいです',
 22: 'おかしくないですか',
 23: '納得できない',
 24: '納得できません',
 25: '納得しました',
 26: '納得できました',
 27: '解決しました',
 28: '解決しません',
 29: 'ほっとしました',
 30: '安心です',
 31: '安心しました',
 32: '不安です',
 33: '不安なんです',
 34: '不安がなくなりました',
 35: 'ありがとうございます'}

In [ ]:
folderpathlist = glob.glob("/content/gdrive/MyDrive/yuichiro_voice/*")

In [ ]:
folderpathlist.remove('/content/gdrive/MyDrive/yuichiro_voice/label.txt')

In [ ]:
wavpathlist = []
for eachfolder in folderpathlist:
  wavlist = glob.glob(eachfolder + "/*.wav")
  wavpathlist.extend(wavlist)

In [ ]:
arr = []
for eachpath in wavpathlist:
    filename = get_filename(eachpath)
    name = int(filename.split(".")[0])
    label = pathdict[name]
    arr.append([eachpath, label])

In [ ]:
main = pd.DataFrame(arr, columns =['path', 'sentence'])

In [ ]:
main

,path,sentence
0,/content/gdrive/MyDrive/yuichiro_voice/origina...,お電話ありがとうございます
1,/content/gdrive/MyDrive/yuichiro_voice/origina...,どのようなご用件
2,/content/gdrive/MyDrive/yuichiro_voice/origina...,どのようなお問い合わせ
3,/content/gdrive/MyDrive/yuichiro_voice/origina...,ありがとう
4,/content/gdrive/MyDrive/yuichiro_voice/origina...,申し訳ありません
...,...,...
681,/content/gdrive/MyDrive/yuichiro_voice/pydubm0...,安心です
682,/content/gdrive/MyDrive/yuichiro_voice/pydubm0...,不安です
683,/content/gdrive/MyDrive/yuichiro_voice/pydubm0...,不安なんです
684,/content/gdrive/MyDrive/yuichiro_voice/pydubm0...,不安がなくなりました


In [ ]:
folderpathlist = glob.glob("/content/gdrive/MyDrive/yamaoka_san_voice/*")

In [ ]:
folderpathlist.remove('/content/gdrive/MyDrive/yamaoka_san_voice/label.txt')

In [ ]:
y_arr = []
for eachpath in wavpathlist:
    filename = get_filename(eachpath)
    name = int(filename.split(".")[0])
    label = pathdict[name]
    y_arr.append([eachpath, label])

In [ ]:
wavpathlist = []
for eachfolder in folderpathlist:
  wavlist = glob.glob(eachfolder + "/*.wav")
  wavpathlist.extend(wavlist)

In [ ]:
y_main = pd.DataFrame(y_arr, columns =['path', 'sentence'])

In [ ]:
y_main

,path,sentence
0,/content/gdrive/MyDrive/yuichiro_voice/origina...,お電話ありがとうございます
1,/content/gdrive/MyDrive/yuichiro_voice/origina...,どのようなご用件
2,/content/gdrive/MyDrive/yuichiro_voice/origina...,どのようなお問い合わせ
3,/content/gdrive/MyDrive/yuichiro_voice/origina...,ありがとう
4,/content/gdrive/MyDrive/yuichiro_voice/origina...,申し訳ありません
...,...,...
681,/content/gdrive/MyDrive/yuichiro_voice/pydubm0...,安心です
682,/content/gdrive/MyDrive/yuichiro_voice/pydubm0...,不安です
683,/content/gdrive/MyDrive/yuichiro_voice/pydubm0...,不安なんです
684,/content/gdrive/MyDrive/yuichiro_voice/pydubm0...,不安がなくなりました


In [ ]:
#training_df = final_df
#testing_df = final_df.sample(frac = 0.3)

In [ ]:
from datasets import Dataset
common_voice_train = Dataset.from_pandas(y_main)
common_voice_test = Dataset.from_pandas(main)

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path"]))

,sentence
0,解決しません
1,納得しました
2,納得できない
3,そうですね
4,ありがとう
5,いえ
6,納得できました
7,ありがとう
8,不安なんです
9,そうですね


## プリプロセス

### 漢字平仮名　-> ro-maji変換

In [ ]:
import re
chars_to_ignore_regex = '[\、\？\。\！\.\」\「\〇\．\…\（\)\＋\）\.]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [ ]:
from pykakasi import kakasi

def kanachange(batch):
    kka = kakasi()

    kka.setMode('H', 'a')
    kka.setMode('K', 'a')
    kka.setMode('J', 'a')

    conv = kka.getConverter()

    batch["sentence"] = conv.do(batch["sentence"])

    return batch

common_voice_train = common_voice_train.map(kanachange)
common_voice_test = common_voice_test.map(kanachange)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since ver

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path"]))

,sentence
0,arigatougozaimasu
1,kaiketsushimasen
2,konnichiwa
3,soudesune
4,fuanganakunarimashita
5,iie
6,kaiketsushimashita
7,yokattadesu
8,kaiketsushimasen
9,tasukarimashita


### 辞書作成

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)


{'a': 0, 's': 1, 'r': 2, 'y': 3, ' ': 4, 'f': 5, 'e': 6, 'o': 7, 'g': 8, 'z': 9, 'd': 10, 'h': 11, 'w': 12, 'm': 13, 'c': 14, 't': 15, 'i': 16, 'u': 17, 'k': 18, 'n': 19}


In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

22

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
import json
with open('/content/gdrive/MyDrive/roma_vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

### プリプロセス２

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-japanese-YY-demo")

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 8000, 16000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
def resample2(batch):
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
common_voice_train = common_voice_train.map(resample2, num_proc=4)
common_voice_test = common_voice_test.map(resample2, num_proc=4)

In [ ]:
yama_df = common_voice_train.to_pandas()
yui_df = common_voice_test.to_pandas()

In [ ]:
final_df = pd.concat([yama_df, yui_df])

In [ ]:
training_df = final_df
testing_df = final_df.sample(frac = 0.3)

In [ ]:
training_df = training_df.reset_index()
testing_df = testing_df.reset_index()

In [ ]:
common_voice_train = Dataset.from_pandas(training_df)
common_voice_test = Dataset.from_pandas(testing_df)

## 音声サンプリング

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: konnichiwa 
Input array shape: (37894,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8,  batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## Training phase


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

### **CNN** fecure extreact

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-japanese-YY-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=False,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
200,9.753300,2.847992,1.000000,30.541400,13.490000
400,2.251900,0.528370,1.682039,31.020100,13.282000
600,0.218100,0.011237,1.080097,31.070100,13.260000
800,0.053100,0.003015,1.060680,31.318700,13.155000
1000,0.023500,0.002189,0.980583,30.717100,13.413000
1200,0.016000,0.001454,0.985437,30.197900,13.643000


TrainOutput(global_step=1290, training_loss=1.9103615666544715, metrics={'train_runtime': 4543.8055, 'train_samples_per_second': 0.284, 'total_flos': 3.2104075928599777e+18, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 1707996, 'init_mem_gpu_alloc_delta': 1261845504, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1384163, 'train_mem_gpu_alloc_delta': 3779276288, 'train_mem_cpu_peaked_delta': 61378407, 'train_mem_gpu_peaked_delta': 1827236352})

#モデル検証

In [ ]:
from google.colab import drive

/drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


## Testing 

In [ ]:
MODEL_ID = "/content/gdrive/MyDrive/wav2vec2-large-xlsr-japanese-YY-demo/checkpoint-1200"

In [ ]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
import torch
import librosa
from datasets import load_dataset
import glob
import torchaudio
import numpy as np
import json

In [ ]:
file1 = open('/content/gdrive/MyDrive/yamaoka_san_voice/label.txt', 'r')
Lines = file1.readlines()
pathdict = {}
for i in Lines:
    s = i.rstrip('\n').split('.')
    label = s[1].replace(" ", "")
    name = int(s[0])
    pathdict[name] = label

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
import romkan
def toKanji(s):
    s = romkan.to_hiragana(s)
    if s != None:
        return s

###Fathers voice


In [ ]:
pathlist = glob.glob("/content/gdrive/MyDrive/test_father/*.wav") 

In [ ]:
def load_and_resample(file_path):
    signal_tensor, samling_rate = torchaudio.load(file_path)
    signal_array = signal_tensor[0].numpy()
    return signal_array

In [ ]:
signals = []
for eachpath in pathlist:
    signal = load_and_resample(eachpath)
    signal_list = signal.tolist()
    signals.append([signal_list,eachpath])

In [ ]:
predicted = []
for eachsginals, path in signals:
    inputs = processor(eachsginals, sampling_rate=16_000, return_tensors="pt", padding=True)
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    predicted_sentences = processor.batch_decode(predicted_ids)
    predicted.append([predicted_sentences[0].replace("[PAD]", ""),path])


In [ ]:
for predsent, path in predicted:
    filename = get_filename(path)
    name = int(filename.split(".")[0])
    label = pathdict[name]
    print("Reference: " + label)
    print("Roma-ji Prediction: " + predsent)
    print("kana translated: "+ toKanji(predsent))
    print("------------------------------------------------------------------------------")
    

Reference: お電話ありがとうございます
Roma-ji Prediction: odenwarigatougozaimasu
kana translated: おでんわりがとうございます
------------------------------------------------------------------------------
Reference: どのようなご用件
Roma-ji Prediction: donoyounagoyouken
kana translated: どのようなごようけん
------------------------------------------------------------------------------
Reference: どのようなお問い合わせ
Roma-ji Prediction: donoyounaotoiawase
kana translated: どのようなおといあわせ
------------------------------------------------------------------------------
Reference: ありがとう
Roma-ji Prediction: arigatou
kana translated: ありがとう
------------------------------------------------------------------------------
Reference: はい
Roma-ji Prediction: hai
kana translated: はい
------------------------------------------------------------------------------
Reference: いいえ
Roma-ji Prediction: iie
kana translated: いいえ
------------------------------------------------------------------------------
Reference: 申し訳ありません
Roma-ji Prediction: takinarimasen
kana tran

In [ ]:
keyaudiopath = '/content/gdrive/MyDrive/originalCallcenterAudio/man1.wav'

In [ ]:
  signal = load_and_resample(keyaudiopath)
  signal_list = signal.tolist()
  inputs = processor(signal_list, sampling_rate=16_000, return_tensors="pt", padding=True)
  logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  predicted_sentences = processor.batch_decode(predicted_ids)
  predicted_sentencesver = predicted_sentences[0].replace("[PAD]", "")
  print("Roma-ji Prediction: " + predicted_sentencesver)
  print("kana translated: "+ toKanji(predicted_sentencesver))



Roma-ji Prediction: ohayougozaimatoukewainateorimasu
kana translated: おはようございまとうけわいなております


###Yamaoka_san Voice


In [ ]:
def load_and_resample(file_path, resamling_rate=16000):
    signal_tensor, samling_rate = torchaudio.load(file_path)
    signal_array = signal_tensor[0].numpy()
    #signal_array = librosa.resample(signal_array, 16000, resamling_rate)
    return signal_array

In [ ]:
pathlist = glob.glob("/content/gdrive/MyDrive/yamaoka_san_voice/original/*.wav") 

In [ ]:
signals = []
for eachpath in pathlist:
    signal = load_and_resample(eachpath)
    signal_list = signal.tolist()
    signals.append([signal_list,eachpath])

KeyboardInterrupt: ignored

In [ ]:
predicted = []
for eachsginals, path in signals:
    inputs = processor(eachsginals, sampling_rate=16_000, return_tensors="pt", padding=True)
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    predicted_sentences = processor.batch_decode(predicted_ids)
    predicted.append([predicted_sentences[0].replace("[PAD]", ""),path])


In [ ]:
for predsent, path in predicted:
    filename = get_filename(path)
    name = int(filename.split(".")[0])
    label = pathdict[name]
    print("Reference: " + label)
    print("Roma-ji Prediction: " + predsent)
    print("kana translated: "+ toKanji(predsent))
    print("------------------------------------------------------------------------------")

###Callcenter test


In [ ]:
  keyaudiopath = '/content/gdrive/MyDrive/originalCallcenterAudio/man1.wav'
  signal = load_and_resample(keyaudiopath)
  signal_list = signal.tolist()
  inputs = processor(signal_list, sampling_rate=16_000, return_tensors="pt", padding=True)
  logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  predicted_sentences = processor.batch_decode(predicted_ids)
  predicted_sentencesver = predicted_sentences[0].replace("[PAD]", "")
  print("Roma-ji Prediction: " + predicted_sentencesver)
  print("kana translated: "+ toKanji(predicted_sentencesver))

NameError: ignored

###Shogos Voice

In [ ]:
def load_and_resample(file_path, resamling_rate=16000):
    signal_tensor, samling_rate = torchaudio.load(file_path)
    signal_array = signal_tensor[0].numpy()
    #signal_array = librosa.resample(signal_array, 16000, resamling_rate)
    return signal_array

In [ ]:
pathlist = glob.glob("/content/gdrive/MyDrive/shogo_voice/*.wav") 

In [ ]:
signals = []
for eachpath in pathlist:
    signal = load_and_resample(eachpath)
    signal_list = signal.tolist()
    signals.append([signal_list,eachpath])

In [ ]:
predicted = []
for eachsginals, path in signals:
    inputs = processor(eachsginals, sampling_rate=16_000, return_tensors="pt", padding=True)
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    predicted_sentences = processor.batch_decode(predicted_ids)
    predicted.append([predicted_sentences[0].replace("[PAD]", ""),path])


In [ ]:
for predsent, path in predicted:
    filename = get_filename(path)
    name = int(filename.split(".")[0])
    label = pathdict[name]
    print("Reference: " + label)
    print("Roma-ji Prediction: " + predsent)
    print("kana translated: "+ toKanji(predsent))
    print("------------------------------------------------------------------------------")

Reference: 不安なんです
Roma-ji Prediction: fuannandesu
kana translated: ふあんあんです
------------------------------------------------------------------------------
Reference: 助かりました
Roma-ji Prediction: tasukarimashita
kana translated: たすかりました
------------------------------------------------------------------------------
Reference: 安心しました
Roma-ji Prediction: anshinshimashita
kana translated: あんしんしました
------------------------------------------------------------------------------
Reference: 納得しました
Roma-ji Prediction: nattokushimashita
kana translated: なっとくしました
------------------------------------------------------------------------------
Reference: 安心です
Roma-ji Prediction: anshindesu
kana translated: あんしんです
------------------------------------------------------------------------------
Reference: 納得できません
Roma-ji Prediction: nattokudekimasen
kana translated: なっとくできません
------------------------------------------------------------------------------
Reference: 納得できました
Roma-ji Prediction: nattokudekimashi

###Suwa-san voice

In [ ]:
def load_and_resample(file_path, resamling_rate=16000):
    signal_tensor, samling_rate = torchaudio.load(file_path)
    signal_array = signal_tensor[0].numpy()
    #signal_array = librosa.resample(signal_array, 16000, resamling_rate)
    return signal_array

In [ ]:
pathlist = glob.glob("/content/gdrive/MyDrive/training_demo/*.wav") 

In [ ]:
signals = []
for eachpath in pathlist:
    signal = load_and_resample(eachpath)
    signal_list = signal.tolist()
    signals.append([signal_list,eachpath])

In [ ]:
predicted = []
for eachsginals, path in signals:
    inputs = processor(eachsginals, sampling_rate=16_000, return_tensors="pt", padding=True)
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    predicted_sentences = processor.batch_decode(predicted_ids)
    predicted.append([predicted_sentences[0].replace("[PAD]", ""),path])


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


###suwa-san cut *voice*


In [ ]:
def load_and_resample(file_path, resamling_rate=16000):
    signal_tensor, samling_rate = torchaudio.load(file_path)
    signal_array = signal_tensor[0].numpy()
    #signal_array = librosa.resample(signal_array, 16000, resamling_rate)
    return signal_array

In [ ]:
  keyaudiopath = '/content/gdrive/MyDrive/training_demo/suwa.wav'
  signal = load_and_resample(keyaudiopath)
  signal_list = signal.tolist()
  inputs = processor(signal_list, sampling_rate=16_000, return_tensors="pt", padding=True)
  logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  predicted_sentences = processor.batch_decode(predicted_ids)
  predicted_sentencesver = predicted_sentences[0].replace("[PAD]", "")
  print("Roma-ji Prediction: " + predicted_sentencesver)
  print("kana translated: "+ toKanji(predicted_sentencesver))

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Roma-ji Prediction: odenwaarigatougozaimasudonogagoyoukendonogaotoiwasearigatoumoshiakearimasenkaiiiekongotonoyoroshikunaitashimasukakeshimashitakaigeshimasenyokattadesuhontounioseainatteorimasu
kana translated: おでんわありがとうございますどのがごようけんどのがおといわせありがとうもしあけありませんかいいいえこんごとのよろしくないたしますかけしましたかいげしませにょかったですほんとうにおせあいなっております


### suwa-san voice cut difmodel


In [ ]:
MODEL_ID = "/content/gdrive/MyDrive/wav2vec2-large-xlsr-japanese-rome-v1-demo"

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
  keyaudiopath = '/content/gdrive/MyDrive/training_demo/suwa.wav'
  signal = load_and_resample(keyaudiopath)
  signal_list = signal.tolist()
  inputs = processor(signal_list, sampling_rate=16_000, return_tensors="pt", padding=True)
  logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  predicted_sentences = processor.batch_decode(predicted_ids)
  predicted_sentencesver = predicted_sentences[0].replace("[PAD]", "")
  print("Roma-ji Prediction: " + predicted_sentencesver)
  print("kana translated: "+ toKanji(predicted_sentencesver))

Roma-ji Prediction: odenhaarigatoozaimadonorenagoyokendonoeraotoiwasearigatomoshakiarimasenhaiiiekogotomoyoroshikunoeitashimakaketshimashitatakyashimasenyokattadesukontoniosaranattorimasu
kana translated: おでんはありがとおざいまどのれなごよけんどのえらおといわせありがともしゃきありませんはいいいえこごともよろしくのえいたしまかけtしましたたきゃしませにょかったですこんとにおさらなっとります
